![Cirq](https://github.com/quantumlib/cirq/raw/master/docs/Cirq_logo_color.svg?sanitize=true=)

[Cirq](https://github.com/quantumlib/cirq) is a framework for writing quantum algorithms for noisy intermediate scale quantum (NISQ) devices. Roughly speaking, NISQ devices are those with O(100) qubits that can enact O(1000) gates.  Because the resources for NISQ devices are so constrained we believe that a framework for writing programs on these devices needs to be aware of all of the architectural properties of the device on which the algorithm is written. This is in contrast to other frameworks where there is a clean separation between the abstract model being used and the details of the device.  

In this tutorial we will cover the basics of writing quantum algorithms in Cirq.

---



>>>[Core Concepts: Qubits, Moments, Operations, and Circuits](#scrollTo=8A7a3jcql1l5)

>>>[Create a Circuit](#scrollTo=VFwmWPf7D057)

>>>[Building Circuits (InsertStrategy)](#scrollTo=uaDb6B_jPgrb)

>>>[Exercise: Create a circuit](#scrollTo=y9conKPAPn26)

>>>[Simulate a Circuit](#scrollTo=X15yPl_KQ20Z)

>>>[Parameterized Circuits](#scrollTo=3HtlMxa6QpVo)

>>>[Rabi-flop experiment](#scrollTo=r-CjbPwkRI_I)

>>>[Devices](#scrollTo=PvJCA3e0QsuI)

>>>[Compiling / Optimizing](#scrollTo=J9ia4eatUQ_x)

>>>[Exercise: Simplify flipped CNOTs](#scrollTo=--aUfkiaUb3S)



In [1]:
!pip install cirq

To verify that Cirq is installed in your environment, try to `import cirq` and print out a diagram of the Foxtail device. It should produce a 2x11 grid of qubits.

In [2]:
import cirq
print(cirq.google.Foxtail)

(0, 0)───(0, 1)───(0, 2)───(0, 3)───(0, 4)───(0, 5)───(0, 6)───(0, 7)───(0, 8)───(0, 9)───(0, 10)
│        │        │        │        │        │        │        │        │        │        │
│        │        │        │        │        │        │        │        │        │        │
(1, 0)───(1, 1)───(1, 2)───(1, 3)───(1, 4)───(1, 5)───(1, 6)───(1, 7)───(1, 8)───(1, 9)───(1, 10)


...

...

(pause for everyone to get this working)

...

...

Be aware that Cirq is still alpha software, meaning **we are still making breaking changes all the time**. If you don't want your project to suddenly go from working to not working when we release a new version, you should depend on a *specific version* of Cirq and periodically bump that version to the latest one. For the purposes of this tutorial, we are just using the latest version.

### Core Concepts: Qubits, Moments, Operations, and Circuits
 
 In Cirq, circuits are represented either by a `Circuit` object or a `Schedule` object. `Schedule`s offer more control over quantum gates and circuits at the timing level.

Conceptually: a `Circuit` is a collection of `Moment`s. A `Moment` is a collection of `Operation`s that all act during the same abstract time slice. An `Operation` is a an effect that operates on a specific subset of Qubits. The most common type of `Operation` is a `Gate` applied to several qubits (a "`GateOperation`"). The following diagram should help illustrate these concepts.

![Circuits, Moments, and Operations.](https://cirq.readthedocs.io/en/latest/_images/CircuitMomentOperation.png)

### Create a Circuit

Let's create a `Circuit`.  Note that in the installing section we imported cirq, so we will assume that cirq has been imported throughout the rest of this notebook.

In [3]:
a = cirq.NamedQubit("a")
b = cirq.NamedQubit("b")
c = cirq.NamedQubit("c")
ops = [cirq.H(a), cirq.H(b), cirq.CNOT(b, c), cirq.H(b)]
circuit = cirq.Circuit.from_ops(ops)
print(circuit)

a: ───H───────────

b: ───H───@───H───
          │
c: ───────X───────


We can unpack this a bit and see all of the components for the circuit.


The first thing we do is pick some qubits to use. There are many different types of qubits in Cirq, and you can define your own by inheriting from the `cirq.QubitId` class. There's nothing inherently special or magical about the qubit types. They simply identify what you wish to operate on, which is relevant when you are targeting a specific device. For example, if we were creating a circuit for the FoxTail device we would use `cirq.GridQubit(0, 0)` to refer to the qubit in the top-left position of the device. To keep these simple for now, we'll start with abstract qubits simply identified by a name such as "a".
```
a = cirq.NamedQubit("a")
```

Next we encounter the object `cirq.H`, which is a Hadamard gate.  `cirq.H` is an instance of the `cirq.HGate` class, which itself is a subclass of `Gate` (along with other classes). 
$$H = {1 \over \sqrt{2}} \left[ \begin{array}[cc]  & 1 & 1  \\ 1 & -1 \end{array}\right]$$

`Gate` objects have the ability to applied "on" to one or more qubits.  There are two ways to do this for gates, either using the `on` method or by directly calling the gate on the qubits as if the gate were a function and the qubits were arguments.  For example to apply the `H` onto qubit `a` we can say
```
cirq.H.on(a)
```
or 
```
cirq.H(a)
```

The result of those expressions is a `GateOperation` object, which is a type of `Operation`. `Operation`s are what we use to make up a `Circuit`. Once you have a collection of operations, you can construct a `Circuit` using the class method `Circuit.from_ops` (more on that in a minute):
```
circuit = cirq.Circuit.from_ops(ops)
```
The last thing we did in the example code was use the (surprisingly useful) ability to print the circuit as a text diagram.

The diagram is visually helpful, but it doesn't really get into the internal details of how the `Circuit` is represented. 
A `Circuit` is made up of a sequence of `Moment` objects.
And each `Moment` object is a list of non-overlapping `Operation`s.
To see this internal structure, we can iterate over the `Moment`s in the `Circuit` while printing them out.

In [4]:
for i, moment in enumerate(circuit):
    print('Moment {}: {}'.format(i, moment))

Moment 0: H(a) and H(b)
Moment 1: CNOT(b, c)
Moment 2: H(b)


We can also just print the circuit's `repr`, which returns a somewhat more detailed (if less readable) expression.

In [6]:
print(repr(circuit))

Circuit([
    Moment((GateOperation(H, (NamedQubit('a'),)), GateOperation(H, (NamedQubit('b'),)))),
    Moment((GateOperation(CNOT, (NamedQubit('b'), NamedQubit('c'))),)),
    Moment((GateOperation(H, (NamedQubit('b'),)),))])


The usefulness of printing the `repr` is that it includes *all* the gory details.
In fact, it is valid python code that will evaluate to exactly the circuit we created.
(... Except that it's missing the prefix "`cirq.`" everywhere.
That will be fixed in the next version.
Also it will be a bit more compact; using `G(q)` instead of `GateOperation(G, (q,))`.)

### Building Circuits (InsertStrategy)

Above we created the `Circuit` using `from_ops`.  But there are many ways to construct and modify circuits, and each of these is useful in different contexts.  Here are a few examples:


1.   `from_ops`: This is the simplest way to make a circuit. Give this method some operations, and out pops a circuit.
2.  `append`:  `Circuit`s are mutable. You can start with an empty `c = cirq.Circuit()` and simply `c.append(operations)` to add on more and more operations 
3. `insert`:  Instead of appending, you can insert before a particular moment location (labeled by an integer index)
4.  By using `Circuit`'s constructor, which takes a list of `Moment`s. Each `Moment` must be explicitly constructed with its own list of `Operation`s. This is tedious, but gives complete control over how the operations are layed out.

You may have noticed that there is a hole in what we've explained so far.
`from_ops` effectively takes a 1-dimensional sequence of operations, but the output is a 2-dimensional circuit (a list-of-lists-of-operations).
There is a degree of freedom that hasn't been account for: how does cirq choose the moment that each operation will be placed within?
The answer is: it depends on the  `InsertStrategy` you choose.

There are currently four insertion strategies in Cirq:

1.  `InsertStrategy.EARLIEST`
2. `InsertStrategy.NEW`,
3. `InsertStrategy.INLINE`
4. `InsertStrategy.NEW_THEN_INLINE` (currently the default)

`InsertStrategy.EARLIEST` is defined as
> `InsertStrategy.EARLIEST`: Scans backward from the insert
> location until a moment with operations touching qubits affected by the
> operation to insert is found. The operation is added into the moment just
> after that location.

For example, if we first create an `Operation` in a single moment,
and then use `InsertStrategy.EARLIEST` the `Operation` can slide back to this
first `Moment` if there is space

An `InsertStrategy` defines how ``Operations`` are placed in a `Circuit` when requested to be inserted at a given location.
Here a `location` is identified by the index of the `Moment` in the `Circuit` that operations should be placed before (in the case of `Circuit.append` this means inserting at the index `len(circuit)`; which is one more than the largets moment index and so represents the end of the circuit).

Let's define a method that prints out a circuit as it is being built with a specified strategy, and see what happens when we use the `EARLIEST` strategy.

In [7]:
def make_with_strategy(strategy):
    ops = [cirq.X(b), cirq.CZ(b, c), cirq.Y(c), cirq.Z(a)]
    circuit = cirq.Circuit()
    print("strategy:", strategy)
    for op in ops:
        circuit.append(op, strategy=strategy)
        print("\nafter appending", op)
        print(circuit)
        
make_with_strategy(cirq.InsertStrategy.EARLIEST)

strategy: EARLIEST

after appending X(b)
b: ───X───

after appending CZ(b, c)
b: ───X───@───
          │
c: ───────@───

after appending Y(c)
b: ───X───@───────
          │
c: ───────@───Y───

after appending Z(a)
a: ───Z───────────

b: ───X───@───────
          │
c: ───────@───Y───


Note how the `Z` gate ended up happening before the `CZ` gate, even though it was appended last.
Whereas the `Y` gate could not move past the `CZ` and so was placed in the moment after the `CZ`.

Contrast this with the `InsertStrategy.NEW` `InsertStrategy`:
> `InsertStrategy.NEW`: Every operation that is inserted is
> created in a new moment.


In [8]:
make_with_strategy(cirq.InsertStrategy.NEW)

strategy: NEW

after appending X(b)
b: ───X───

after appending CZ(b, c)
b: ───X───@───
          │
c: ───────@───

after appending Y(c)
b: ───X───@───────
          │
c: ───────@───Y───

after appending Z(a)
a: ───────────────Z───

b: ───X───@───────────
          │
c: ───────@───Y───────


Here every operator processed by the append ends up in a new moment.
`InsertStrategy.NEW` is most useful when you are inserting a single operation and
don't want it to get mixed into other ``Moments``.

Another strategy is `InsertStrategy.INLINE`:
> `InsertStrategy.INLINE`: Attempts to add the operation to
> insert into the moment just before the desired insert location. But, if
> there's already an existing operation affecting any of the qubits touched
> by the operation to insert, a new moment is created instead.

In [9]:
make_with_strategy(cirq.InsertStrategy.INLINE)

strategy: INLINE

after appending X(b)
b: ───X───

after appending CZ(b, c)
b: ───X───@───
          │
c: ───────@───

after appending Y(c)
b: ───X───@───────
          │
c: ───────@───Y───

after appending Z(a)
a: ───────────Z───

b: ───X───@───────
          │
c: ───────@───Y───


The `Z` gate ends up in the final moment because there is room, but is not moved backwards any further than that. `INLINE` is useful when you want your operations to stay roughly where you put them, but to pack a little better than `NEW`.

The final strategy, and currently the default (though this is likely to change) is `NEW_THEN_INLINE`. If you are appending operations one at a time, `NEW_THEN_INLINE` behaves exactly like `NEW`. But `append` doesn't just take individual operations, it also takes lists of operations. And when you specify a list, `NEW_THEN_INLINE` will insert the first item in a new moment and then inline later operations into that moment. It's useful when you want to keep operations from the same append near each other in the circuit.

Note that `from_ops`, `append`, and `insert` don't just take indivual operations and lists of operations.
They take anything that can be iteratively flattened into a sequence of operations.
You can call these methods with some pretty complicated stuff, and it will just work.
For example, let's make a generator function that yields a generator function yielding operations and tuples of operations and pass that into one of the `Circuit` methods.

In [10]:
def xor_swap(a, b):
  yield cirq.CNOT(a, b), cirq.CNOT(b, a)
  yield cirq.CNOT(a, b)

def left_rotate(qubits):
    for i in range(len(qubits) - 1):
        a, b = qubits[i:i+2]
        yield xor_swap(a, b)

line = cirq.LineQubit.range(5)
print(cirq.Circuit.from_ops(left_rotate(line)))

0: ───@───X───@───────────────────────────────────────
      │   │   │
1: ───X───@───X───@───X───@───────────────────────────
                  │   │   │
2: ───────────────X───@───X───@───X───@───────────────
                              │   │   │
3: ───────────────────────────X───@───X───@───X───@───
                                          │   │   │
4: ───────────────────────────────────────X───@───X───


The ability to create circuits with generators is very convenient. It allows you to modularize your circuit-creation code into functions in a natural pythonic way.


### Exercise: Create a circuit

Now that you've learned about `InsertStrategy`, here is an exercise to validate your understanding.  Create the following circuit using as few `append` calls as possible.

```
a: ───@───H───────────H───H───
      │
b: ───@───────H───@───H───────
                  │
c: ───H───────────@───────────
```

(A note of caution: printing a circuit in Cirq does not always print a momeent by moment structure. In particular, if a moment has operations that would overlap in the diagram, an extra column is introduced to avoid the overlap. But here imagine that you want exactly the moments indicated by the spacing of the circuit.)

In [11]:
circuit = cirq.Circuit()

# Modify this append call, and add more append calls, to produce the above circuit
circuit.append([])
#1st cluster with earliest
#2nd with New-Inline Strategy
...

print(circuit)

### Simulate a Circuit

Now that you know how to construct a `Circuit` in Cirq, let's use Cirq to simulate the circuit.

Here is a simple circuit

In [12]:
def basic_circuit(meas=True):
    sqrt_x = cirq.X**0.5
    yield sqrt_x(a), sqrt_x(b)
    yield cirq.CZ(a, b)
    yield sqrt_x(a), sqrt_x(b)
    if meas:
        yield cirq.measure(a, b)
        
circuit = cirq.Circuit.from_ops(basic_circuit())
print(circuit)

a: ───X^0.5───@───X^0.5───M───
              │           │
b: ───X^0.5───@───X^0.5───M───


There are a few things to note here.  

First, we are using power (`**`) notation for the first time.
`cirq.X**0.5` is the square root of X gate (specifically: the square root you get when using the right hand rule).
This notation allows you to specify partial rotations without having to multiply by `np.pi` everywhere.
You can also raise an operation to a power after applying it to a qubit, such as `cirq.X(a)**0.5` instead of `(cirq.X**0.5).on(a)`.

Second, we have included a multi-qubit measurement gate in our circuit.
When we run the circuit, we will be given sample results for this measurement.

Now we can simulate this circuit using `cirq.google.XmonSimulator`.

In [15]:
simulator = cirq.google.XmonSimulator()
result = simulator.run(circuit, repetitions=15)
print(result)

a,b=100111100000000, 111000110000100


Running this multiple times will result in different measurement results each time, since the above circuit produces a superposition over all computational basis states.  

Above we used the `run` method on the simulator. The `run` methods mimic the actual hardware in that they don't give one access to unphysical objects like the wavefunction.  If one wants to get the wave function, then the `simulate` methods can do this:


In [16]:
import numpy as np
circuit = cirq.Circuit.from_ops(basic_circuit(False))
result = simulator.simulate(circuit, qubit_order=[a, b]).final_state

print(np.around(result, 3))

[-0.5-0.j   0. -0.5j  0. -0.5j -0.5+0.j ]


There is another way to simulate circuits, by calling `circuit.apply_unitary_effect_to`. Currently, ironically, this method is faster than the dedicated simulator class. That will change in the future.

In [17]:
result2 = circuit.apply_unitary_effect_to_state(initial_state=0b000)
print(np.around(result2, 3))

[0.5+0.j  0. +0.5j 0. +0.5j 0.5+0.j ]


Note that the results from the two simulators have differing global phase, but are otherwise equivalent. This is because the `XmonSimulator` is automatically decomposing the gates we used into the gate set available on the actual quantum hardware, and the decompositions do not preserve global phase. Cirq provides methods such as `cirq.allclose_up_to_global_phase` to make this easier to deal with.

In [18]:
print(np.allclose(result, result2))
print(cirq.allclose_up_to_global_phase(result, result2))


False
True


Notice that we passed a qubit_order into the `simulate` method.  This order defines the order of the kronecker product used in the resulting `final_state` vector. The `qubit_order` argument is optional. When it is omitted, qubits are ordered ascending by their name (i.e. what their `__str__` method returns).

The simplest `qubit_order` value you can provide is a list of the qubits in the desired ordered. Any qubits from the circuit  that are not in the list will be ordered using the  default `__str__` ordering, but come after qubits that are in the list. Be aware that all qubits in the list are included in the simulation, even if they are not operated on by the circuit.

The mapping from the order of the qubits to the order of the  amplitudes in the wave function can be tricky to understand. 
Basically, it is the same as the ordering used by `numpy.kron`:

In [19]:
first = [1, 1000]
second = [1, 2]
print(np.kron(first, second))

[   1    2 1000 2000]


More concretely, the `k`'th amplitude in the wave function
will correspond to the `k`'th case that would be encountered 
when nesting loops over the possible values of each qubit.
The first qubit's computational basis values are looped over
in the outermost loop, the last qubit's computational basis
values are looped over in the inner-most loop, etc:

In [0]:
i = 0
for first in [0, 1]:
    for second in [0, 1]:
        print('amps[{}] is for first={}, second={}'.format(i, first, second))
        i += 1

amps[0] is for first=0, second=0
amps[1] is for first=0, second=1
amps[2] is for first=1, second=0
amps[3] is for first=1, second=1


The value returned by `simulator.run` is a `TrialResult`, which has utility methods such as `histogram`. `histogram` returns a python `Counter` saying how often various values were sampled.

In [0]:
circuit = cirq.Circuit.from_ops(basic_circuit())
result = simulator.run(circuit, qubit_order=[a, b], repetitions=1000)
print(result.histogram(key='a,b'))

Counter({2: 272, 3: 267, 0: 231, 1: 230})


### Parameterized Circuits

In addition to circuit gates with fixed values, Cirq also  supports gates which can have `Symbol` value. These are values that can be resolved at *run-time*. For simulators these values are resolved by providing a `ParamResolver`.  A `ParamResolver` provides a map from the `Symbol`'s name to its assigned value.

In [0]:
val = cirq.Symbol('s')
x_s = cirq.google.ExpWGate(half_turns=val)
circuit = cirq.Circuit.from_ops(x_s(a), x_s(b))
print(circuit)
print('\nResults')
for y in range(5):
    resolver = cirq.ParamResolver({'s': y / 4.0})
    result = simulator.simulate(circuit, resolver)
    print('s={}: {}'.format(y, np.around(result.final_state, 2)))

a: ───X^s───

b: ───X^s───

Results
s=0: [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
s=1: [ 0.85+0.j    0.  -0.35j  0.  -0.35j -0.15+0.j  ]
s=2: [ 0.5+0.j   0. -0.5j  0. -0.5j -0.5+0.j ]
s=3: [ 0.15+0.j    0.  -0.35j  0.  -0.35j -0.85+0.j  ]
s=4: [ 0.+0.j  0.-0.j  0.-0.j -1.+0.j]


Here we see that the `Symbol` is used in two gates, and then the resolver
provide this value at run time.

Parameterized values are most useful in defining what we call a `Study`.  A `Study` is a collection of trials, where each  trial is a run with a particular set of configurations and which may be run repeatedly.  Running a study returns one  `TrialContext` and `TrialResult` per set of fixed parameter values and repetitions (which are reported as the ``repetition_id`` in the ``TrialContext`` object).  Example:


In [0]:
resolvers = [cirq.ParamResolver({'s': y / 2.0}) for y in range(3)]
circuit = cirq.Circuit()
circuit.append([x_s(a), x_s(b)])
circuit.append([cirq.measure(a), cirq.measure(b)])
results = simulator.run_sweep(program=circuit,
                              params=resolvers,
                              repetitions=20)
for result in results:
    print('for s = {}:\n{}\n'.format(result.params['s'], result))


for s = 0.0:
a=00000000000000000000
b=00000000000000000000

for s = 0.5:
a=11001110010000010110
b=10101010111000011001

for s = 1.0:
a=11111111111111111111
b=11111111111111111111



where we see that different runs for the case that the
qubit has been rotated into a superposition over computational
basis states yield different measurement results per run.
Also note that we now see the use of the ``TrialContext`` returned as the first
tuple from ``run``: it contains the ``param_dict`` describing what values were
actually used in resolving the ``Symbol``s.

Above we passed in a list of `ParamResolver`s to the `params` parameter of `run_sweep`.  But one can also pass in a `Sweepable`.  There are some useful methods for generating `Sweepable`s, for example to generate an equally spaced set of param resolvers one can use `Linspace`


In [0]:
linspace = cirq.Linspace(start=0, stop=1.0, length=11, key='x')
for p in linspace:
    print(p)

ParamResolver(OrderedDict([('x', 0.0)]))
ParamResolver(OrderedDict([('x', 0.1)]))
ParamResolver(OrderedDict([('x', 0.2)]))
ParamResolver(OrderedDict([('x', 0.3)]))
ParamResolver(OrderedDict([('x', 0.4)]))
ParamResolver(OrderedDict([('x', 0.5)]))
ParamResolver(OrderedDict([('x', 0.6)]))
ParamResolver(OrderedDict([('x', 0.7)]))
ParamResolver(OrderedDict([('x', 0.8)]))
ParamResolver(OrderedDict([('x', 0.9)]))
ParamResolver(OrderedDict([('x', 1.0)]))


### Devices

NISQ algorithms work in a regime where every gate counts.  A key philosophy behind Cirq is that we believe the details of the hardware, the performance characteristics, as well as device constraings, will be key to getting the most out of NISQ algorithms.  Towards this end these hardware features are contained in the `Device` class.

For example, here is Google's Bristlecone device

In [0]:
print(cirq.google.Bristlecone)

                                             (0, 5)────(0, 6)
                                             │         │
                                             │         │
                                    (1, 4)───(1, 5)────(1, 6)────(1, 7)
                                    │        │         │         │
                                    │        │         │         │
                           (2, 3)───(2, 4)───(2, 5)────(2, 6)────(2, 7)───(2, 8)
                           │        │        │         │         │        │
                           │        │        │         │         │        │
                  (3, 2)───(3, 3)───(3, 4)───(3, 5)────(3, 6)────(3, 7)───(3, 8)───(3, 9)
                  │        │        │        │         │         │        │        │
                  │        │        │        │         │         │        │        │
         (4, 1)───(4, 2)───(4, 3)───(4, 4)───(4, 5)────(4, 6)────(4, 7)───(4, 8)───(4, 9)───(4, 10)
         │        │      

`Device`s also contain more information about the timing of the device.  For example here we can calculate the duration of an `Exp11Gate` on the `Bristlecone` device

In [0]:
brissy = cirq.google.Bristlecone
op = cirq.google.Exp11Gate().on(cirq.GridQubit(5, 5))
print(brissy.duration_of(op))

50ns


Note that the current duration information is just arbitrary numbers I entered for testing purposes. They are the right order of magnitude, but very much subject to change.

Another property of devices is that they can be used to enforce constraints from the hardware, both checking that these constraints are satisfied, but also enforcing the constraints on the device.  For example on the `Bristlecone` device a two qubit gate has the property that one cannot simulatenously perform a two qubit gate that acts on the adjacent qubits.  So for example if we create such a `Circuit` and validate it using the device it will yell at us

In [0]:
q55 = cirq.GridQubit(5, 5)
q56 = cirq.GridQubit(5, 6)
q66 = cirq.GridQubit(6, 6)
q67 = cirq.GridQubit(6, 7)
ops = [cirq.google.Exp11Gate()(q55, q56), cirq.google.Exp11Gate()(q66, q67)]
circuit = cirq.Circuit.from_ops(ops)
cirq.google.Bristlecone.validate_circuit(circuit)

ValueError: ignored

But more interestingly we could have passed the device into the `Circuit` and it will perform the creation of the circuit (using the insertion semanics as described above) such that the device cannot violate the constraints.

In [0]:
ops = [cirq.google.Exp11Gate().on(q55, q56), cirq.google.Exp11Gate().on(q66, q67)]
circuit = cirq.Circuit(device=cirq.google.Bristlecone)
circuit.append(ops)
print(circuit)

(5, 5): ───@───────
           │
(5, 6): ───@───────

(6, 6): ───────@───
               │
(6, 7): ───────@───


### Compiling / Optimizing

Cirq includes several circuit optimizers. The simplest way to apply these optimizers is with the `cirq.google.optimized_for_xmon` method, which takes a Circuit and spits out a Circuit that will run on Xmon hardware while attempting to improve the circuit.

More generally, it is possible to define your own optimization passes. One particularly useful class for doing so is `PointOptimizer`. A `PointOptimizer` will go over the various operations in a circuit, checking whether a custom method says to do nothing or else to make a change. For example here is a `PointOptimizer` that replaces an `X` gate followed by a `Z` gate with a `Y` gate

In [0]:
class XZOptimizer(cirq.PointOptimizer):
    """Replaces an X followed by a Z with a Y."""
    
    def optimization_at(self, circuit, index, op):
        # Is the gate an X gate?
        if len(op.qubits) != 1:
            return None
        q = op.qubits[0]
        if op != cirq.X(q):
            return None
        next_op_index = circuit.next_moment_operating_on([q], index + 1)
        if next_op_index is None:
            return None
        next_op = circuit.operation_at(q, next_op_index)
        if next_op != cirq.Z(q):
            return None
        new_op = cirq.Y(q)

        return cirq.PointOptimizationSummary(
            clear_span = next_op_index - index + 1,
            clear_qubits=[q], 
            new_operations=[new_op])
        
opt = XZOptimizer()
circuit = cirq.Circuit.from_ops(cirq.CZ(a, b), cirq.X(a), cirq.Z(a), cirq.CZ(a, b), cirq.X(a))
print('Before\n{}'. format(circuit))
opt.optimize_circuit(circuit)
print('After\n{}'.format(circuit))

Before
a: ───@───X───Z───@───X───
      │           │
b: ───@───────────@───────
After
a: ───@───Y───@───X───
      │       │
b: ───@───────@───────


### Exercise: Simplify flipped CNOTs

Write a PointOptimizer that performs the following simplification:

```
a: ───H───@───H───            a: ───X───
          │           ---->         │
b: ───H───X───H───            b: ───@───
```





In [0]:
class HCNotOptimizer(cirq.PointOptimizer):
    def optimization_at(self, circuit, index, op):

        
        
        # Insert your code here.

        
        
        return cirq.PointOptimizationSummary(
            clear_span=???,
            clear_qubits=???, 
            new_operations=???)

    
opt = HCNotOptimizer()
circuit = cirq.Circuit.from_ops(
    cirq.H(a), cirq.H(b),
    cirq.CZ(a, b),
    cirq.H(a), cirq.H(b),
    cirq.CNOT(a, b),
    cirq.H(a), cirq.H(b),
    cirq.CNOT(a, b),
)
print('Before\n{}'. format(circuit))
opt.optimize_circuit(circuit)
print('After\n{}'.format(circuit))


SyntaxError: ignored

### Bernstein-Vazirani algorithm Example


In [0]:
import random
import cirq

In [0]:
def make_oracle(input_qubits,
                output_qubit,
                secret_factor_bits,
                secret_bias_bit):
    """Gates implementing the function f(a) = a·factors + bias (mod 2)."""

    if secret_bias_bit:
        yield cirq.X(output_qubit)

    for qubit, bit in zip(input_qubits, secret_factor_bits):
        if bit:
            yield cirq.CNOT(qubit, output_qubit)

In [0]:
def make_bernstein_vazirani_circuit(input_qubits, output_qubit, oracle):
    """Solves for factors in f(a) = a·factors + bias (mod 2) with one query."""

    c = cirq.Circuit()

    # Initialize qubits.
    c.append([
        cirq.X(output_qubit),
        cirq.H(output_qubit),
        cirq.H.on_each(input_qubits),
    ])

    # Query oracle.
    c.append(oracle)

    # Measure in X basis.
    c.append([
        cirq.H.on_each(input_qubits),
        cirq.measure(*input_qubits, key='result')
    ])

    return c

In [0]:
def bitstring(bits):
    return ''.join(str(int(b)) for b in bits)

In [24]:
def main():
    qubit_count = 8
    circuit_sample_count = 3

    # Choose qubits to use.
    input_qubits = [cirq.GridQubit(i, 0) for i in range(qubit_count)]
    output_qubit = cirq.GridQubit(qubit_count, 0)

    # Pick coefficients for the oracle and create a circuit to query it.
    secret_bias_bit = random.randint(0, 1)
    secret_factor_bits = [random.randint(0, 1) for _ in range(qubit_count)]
    oracle = make_oracle(input_qubits,
                         output_qubit,
                         secret_factor_bits,
                         secret_bias_bit)
    print('Secret function:\nf(a) = a·<{}> + {} (mod 2)'.format(
        ', '.join(str(e) for e in secret_factor_bits),
        secret_bias_bit))

    # Embed the oracle into a special quantum circuit querying it exactly once.
    circuit = make_bernstein_vazirani_circuit(
        input_qubits, output_qubit, oracle)
    print('Circuit:')
    print(circuit)

    # Sample from the circuit a couple times.
    simulator = cirq.google.XmonSimulator()
    result = simulator.run(circuit, repetitions=circuit_sample_count)
    frequencies = result.histogram(key='result', fold_func=bitstring)
    print('Sampled results:\n{}'.format(frequencies))

    # Check if we actually found the secret value.
    most_common_bitstring = frequencies.most_common(1)[0][0]
    print('Most common matches secret factors:\n{}'.format(
        most_common_bitstring == bitstring(secret_factor_bits)))



if __name__ == '__main__':
    main()

Secret function:
f(a) = a·<0, 1, 0, 1, 1, 0, 1, 1> + 0 (mod 2)
Circuit:
(0, 0): ───────H───────────────────────H───M('result')───
                                           │
(1, 0): ───────H───@───────────────────H───M─────────────
                   │                       │
(2, 0): ───────H───┼───────────────────H───M─────────────
                   │                       │
(3, 0): ───────H───┼───@───────────────H───M─────────────
                   │   │                   │
(4, 0): ───────H───┼───┼───@───────────H───M─────────────
                   │   │   │               │
(5, 0): ───────H───┼───┼───┼───────────H───M─────────────
                   │   │   │               │
(6, 0): ───────H───┼───┼───┼───@───────H───M─────────────
                   │   │   │   │           │
(7, 0): ───────H───┼───┼───┼───┼───@───H───M─────────────
                   │   │   │   │   │
(8, 0): ───X───H───X───X───X───X───X─────────────────────
Sampled results:
Counter({'01011011': 3})
Most common 